In [ ]:
#Adding dependecies
import pandas as pd
import geopandas as gpd
import json

In [ ]:
#importing data set
covid_data=pd.read_csv("data/country_vaccinations.csv")
covid_data.head()

In [ ]:
# % of population in the data set is presented up to date
#finding last value for each country, to find the most update value
population_vacc=covid_data[["country","people_fully_vaccinated_per_hundred"]].drop_duplicates(subset=['country'], keep="last")
population_vacc.head()

In [ ]:
#calulacting the sum of all people fully vaccinated in each country
fully_vacc=covid_data[["country","people_fully_vaccinated"]].drop_duplicates(subset=['country'], keep="last")
fully_vacc.head()

In [ ]:
country_name=covid_data["country"].unique()
iso_code=covid_data["iso_code"].unique()
#Creating new data frame
countries_summary=pd.DataFrame({"Country":country_name,
                                "ISO code": iso_code,
                              "Population vaccinated":population_vacc["people_fully_vaccinated_per_hundred"]})
countries_summary["Number of people fully vaccinated"]=fully_vacc["people_fully_vaccinated"]
countries_summary.to_csv("output/Worldwide_vaccination_summary.csv", index=False, header=True)
countries_summary.head()

In [ ]:
#from UK_Kenya_Analysis file collecting information of people fully vaccinated in UK
final_vacc_number_uk=31449915.0
#from kenya_analysis file collecting number of uk population
uk_population=68391673
population_vacc_uk = (final_vacc_number_uk/uk_population)*100
uk_data={"Country":"UK",
        "ISO code": "Nan",
        "Population vaccinated":population_vacc_uk,
        "Number of people fully vaccinated":final_vacc_number_uk}
countries_summary.append(uk_data,  ignore_index=True)

In [ ]:
#identifying the country with the highest percentaged of population fully vaccinated
descending=countries_summary.sort_values(by=["Population vaccinated"], ascending=False)
descending

In [ ]:
#identifying the country with the lowest percentaged of population fully vaccinated
countries_summary.sort_values(by=["Population vaccinated"])

In [ ]:
countries_summary.loc[countries_summary["Country"]=="United Kingdom"]

In [ ]:
countries_summary.loc[countries_summary["Country"]=="Kenya"]

In [ ]:
shapefile = 'data/shape_file/ne_110m_admin_0_countries.shp'
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns.
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

In [ ]:
countries_summary.info()
countries_summary[countries_summary['ISO code'].isnull()]

In [ ]:
#Merge dataframes gdf and df_vacc
merged = gdf.merge(countries_summary, left_on = 'country_code', right_on = 'ISO code')

In [ ]:
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][9]

#Reverse color order so that dark blue is highest vaccination percentage.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40)

#Define custom tick labels for color bar.
tick_labels = tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '45','45':'50','50':'>50%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Share of people fully vaccinated', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Population vaccinated', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)